In [1]:
import spacy
from spacy.matcher import Matcher

In [2]:
# Загружаем модель языка и создаем объект nlp
nlp = spacy.load('ru_core_news_sm')

In [12]:
#question = 'Какое время открытия торгов Американской биржи?'
question = input("Чем я могу вам помочь?\n")
doc = nlp(question)
#Вывод дерева зависимостей
data = spacy.displacy.render(doc, style='dep', jupyter=True)

Чем я могу вам помочь?
 В какое время открывается Российская биржа?


In [13]:
# Создаем паттерн для вопроса с пустым полем для названия биржи
pattern = [{"LOWER": {"IN": ["во", "в"]},'OP': '?'},
           {"LOWER": {"IN": ["какое", "какие"]},'OP': '?'},
           {"LOWER": {"IN": ["время", "часы"]},'OP': '?'},
           {"LOWER": "сколько",'OP': '?'},
           {"LOWER": {"IN": ["открытие", "закрытие"]},'OP': '?'},
           {'POS': "VERB" ,'OP': '?'},
           {"LOWER": {"IN": ["торги", "торгов"]},'OP': '?'},
           {"LOWER": "на",'OP': '?'},
           {"POS": "ADJ"},
           {"LOWER": {"IN": ["бирже", "биржа", "биржу", "биржи"]}}
]


In [14]:
# Инициализируем Matcher и добавляем паттерн
matcher = Matcher(nlp.vocab)
matcher.add('OPENING_TIME', [pattern])

In [15]:
matches = matcher(doc)
# matches

In [16]:
for match_id, start, end in matches:
    span = doc[start:end]
    print("Найден паттерн:", span.text)

Найден паттерн: В какое время открывается Российская биржа
Найден паттерн: какое время открывается Российская биржа
Найден паттерн: время открывается Российская биржа
Найден паттерн: открывается Российская биржа
Найден паттерн: Российская биржа


In [17]:
# Задаем таблицу с данными о времени работы бирж
birzha_timetable = {
    'российский': {
        'откр': '10:00',
        'закр': '18:00'
    },
    'американский': {
        'откр': '09:30',
        'закр': '16:00'
    },
    'японская': {
        'откр': '06:00',
        'закр': '21:00'
    },'китайский': {
        'откр': '14:00',
        'закр': '23:00'
    }
}

In [18]:
if matches:
    match_id, start, end = matches[0]
    for token in doc:
        if token.pos_ == "ADJ": 
            birzha_name = token.lemma_
            #print(token.text, token.lemma_, token.pos_) 
        if token.pos_ == "VERB":
            state = (token.lemma_)[0:4]
    if birzha_name in birzha_timetable:
        print(f"В {birzha_timetable[birzha_name][state]} по Московскому времени.")
    else:
        print('Извините, информации о времени работы данной биржи нет.')

В 10:00 по Московскому времени.


In [19]:
questions = ['Во сколько (открытие) торгов на () бирже?',
     'Американская биржа открывается в какое время?',
     'В какое время закрывается Российская биржа?',
     'Во сколько закрывается Американская биржа?',
     'Азиатская биржа закрывается во сколько?',
     'Когда закрытие торгов на Европейской бирже?']

In [ ]:
igor-kashirin@mail.ru